<a href="https://colab.research.google.com/github/TruongHieuDEV/MachineLearning/blob/main/Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function 
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from PIL import Image

In [ ]:
img = Image.open('anh.jpg')
new_width = 360
new_height = 480
resized_img = np.array(img.resize((new_width, new_height)))
pixel = []
for i in range(new_height):
  for j in range(new_width):
    pixel.append(resized_img[i, j])
pixel = np.array(pixel).astype(np.uint8)

In [ ]:

class KMean():
  def __init__(self, k, max_iterations):
    self.k = k
    self.max_iterations = max_iterations
    self.all_centroids = []
    self.all_labels = []
  def fit(self, dataSet):
    numFeatures = dataSet.shape[1]
    centroids = self.get_random_centroids(dataSet, numFeatures)
    self.all_centroids.append(centroids)
    self.all_labels.append(None)
    
    iterations = 0
    oldCentroids = None
    while not self.should_stop(oldCentroids, centroids, iterations):
      oldCentroids = centroids
      iterations += 1

      labels = self.get_labels(dataSet, centroids)
      self.all_labels.append(labels)

      centroids = self.get_centroids(dataSet, labels)
      self.all_centroids.append(centroids)
    return centroids
  def get_random_centroids(self, dataSet, numFeatures):
    return dataSet[np.random.choice(dataSet.shape[0], k, replace=False)]
  def get_labels(self, dataSet, centroids):
    return np.argmin(cdist(dataSet, centroids), axis = 1)
  def should_stop(self, oldCentroids, centroids, iterations):
    if iterations > self.max_iterations:
      return True
    return np.all(oldCentroids==centroids)
  def get_centroids(self, dataSet, labels):
    centroids = []
    for i in range(self.k):
      data_i = dataSet[labels==i]
      centroids.append(np.mean(data_i, axis=0))
    return np.array(centroids)

In [ ]:
k = 3
kmean = KMean(k, max_iterations=10)
centroids = kmean.fit(pixel)
pixel_new = pixel.copy()
print(centroids)
labels = kmean.all_labels[-1]
for j in range(k):
  pixel_new[labels==j] = centroids[j]
pos = 0
img_new = np.zeros((new_height, new_width, 3))
for x in range(new_height):
  for y in range(new_width):
    img_new[x][y] = pixel_new[pos]
    pos += 1
img_new = img_new.astype(np.uint8)
img = Image.fromarray(img_new, mode='RGB')
img.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

color = ['green', 'blue', 'orange']
gs = GridSpec(nrows=int(np.sqrt(len(kmean.all_centroids)) + 5), ncols=int(np.sqrt(len(kmean.all_centroids)) ))
plt.subplots_adjust(wspace=0.2, hspace=0.4)
plt.figure(figsize=(100,100))
for i in range(len(kmean.all_centroids)):
  ax = plt.subplot(gs[i])
  if i == 0:
    plt.scatter(pixel[:, 0], pixel[:, 1], s=50, color='red', alpha=0.5)
    centroids_i = kmean.all_centroids[i]
    for j in range(kmean.k):
      plt.scatter(centroids_i[j, 0], centroids_i[j, 1], s=1000, marker='x', color='red')
    plt.title('All points in original dataset')
  else:
    centroids_i = kmean.all_centroids[i]
    labels = kmean.all_labels[i]
    for j in range(kmean.k):
      data_i = pixel[labels==j]
      plt.scatter(data_i[:, 0], data_i[:, 1], s=50, color=color[j], alpha=0.5)
      plt.scatter(centroids_i[j, 0], centroids_i[j, 1], color=color[j], s=100, marker='x')
    plt.title(r'Iteration {}'.format(i))

In [ ]:
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=2, max_iter=10, random_state=0)
kmean.fit(dataset)

In [ ]:
K = 10
losses = []
for k in range(1, K):
  kmean_k = KMeans(n_clusters=k, max_iter=10, random_state=0)
  kmean_k.fit(dataset)
  centroids_k = kmean_k.cluster_centers_
  dist = cdist(dataset, centroids_k)
  loss = np.min(dist, axis=1)
  losses.append(np.sum(loss))

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(1, K), losses, 'rx-', alpha=1)

Chúng ta sẽ sử dụng elbow để tìm ra điểm mà ở độ tốc độ suy giảm của hàm biến dạng sẽ thay đổi nhiều nhất, tức là sau điểm này thì khi cluster tăng lên, hàm biến dạng cũng giảm không đáng kể. Một vấn đề nữa khi chúng ta chọn số cluster quá ít thì một số điểm dữ liễu sẽ bị phân vào cluster mà nó không thuộc về dẫn tới hiện tượng underfiting, còn ngược lại khi số lượng cluster quá nhiều thì kích thước của cluster sẽ nhỏ, dẫn tới overfiting. Do đó Elbow sẽ giúp chúng ta chọn ra một số lượng cluster tối ưu giữa overfiting và underfiting.

Một số hạn chế của Kmean:
  

*   Phải chọn cụm phù hợp do dữ liệu chưa được dán nhãn nên không có thông tin về cụm bằng pp Elbow
*   Nhảy cảm với outliers 
*   Tùy vào cách chọn centroids ban đầu mà thuật toán sẽ phân cụm theo các cách khác nhau
*   Không hội  tụ về quy luật phân chia tổng quát với những bộ dữ liệu phức tạp


